In [17]:
import torch
from torchdiffeq import odeint_adjoint as odeint
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
import time
import sys
sys.path.append('../')
from typing import Tuple, Any

%load_ext autoreload
%autoreload 2


import interflow as itf
import interflow.prior as prior
import interflow.fabrics
import interflow.stochastic_interpolant as stochastic_interpolant
import interflow.coupling_flows as coupling_flows
from interflow.coupling_flows import *
from interflow.realnvp import *
from torch import autograd
from functorch import jacfwd, vmap


if torch.cuda.is_available():
    print('CUDA available, setting default tensor residence to GPU.')
    itf.util.set_torch_device('cuda')
else:
    print('No CUDA device found!')
print(itf.util.get_torch_device())


print("Torch version:", torch.__version__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
No CUDA device found!
cpu
Torch version: 2.7.1


In [5]:
C,H,W = 3,32,32
flow = TimeIndexedAffineFlow(shape=(C,H,W), num_layers=16,
                             img_hidden_channels=96, img_blocks=3).cuda()
x = torch.randn(8, C, H, W, device="cuda")
t = torch.rand(8, device="cuda")

y, logdet = flow(x, t)                       # forward
dydt = flow.dTdt(x, t, method="autograd")    # should work
dxdt_inv = flow.dInvTdt(y, t, method="autograd")


OutOfMemoryError: CUDA out of memory. Tried to allocate 9.00 GiB. GPU 0 has a total capacity of 44.39 GiB of which 6.99 GiB is free. Including non-PyTorch memory, this process has 37.40 GiB memory in use. Of the allocated memory 29.49 GiB is allocated by PyTorch, and 7.41 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [6]:
import torch, math

def fmt(b):  # bytes -> GB
    return f"{b/1024**3:.2f} GB"

device = torch.device("cuda:0")

# 1) Instant snapshot
free, total = torch.cuda.mem_get_info(device)
print("CUDA mem get info  : free", fmt(free), " / total", fmt(total))
print("allocated (active) :", fmt(torch.cuda.memory_allocated(device)))
print("reserved (cached)  :", fmt(torch.cuda.memory_reserved(device)))

# 2) Peak tracker for a code region
torch.cuda.reset_peak_memory_stats(device)
# ... run your forward/backward/derivative call here ...
print("peak allocated     :", fmt(torch.cuda.max_memory_allocated(device)))
print("peak reserved      :", fmt(torch.cuda.max_memory_reserved(device)))

# 3) Full text summary (per device)
print(torch.cuda.memory_summary(device=device, abbreviated=False))


CUDA mem get info  : free 6.99 GB  / total 44.39 GB
allocated (active) : 0.89 GB
reserved (cached)  : 36.90 GB
peak allocated     : 0.89 GB
peak reserved      : 36.90 GB
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 14           |        cudaMalloc retries: 14        |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |    913 MiB |    913 MiB |  72260 MiB |  71347 MiB |
|       from large pool |    873 MiB |    873 MiB |  72165 MiB |  71292 MiB |
|       from small pool |     40 MiB |     40 MiB |     94 MiB |     54 MiB |
|-------------------------------------------------

In [21]:
example_usage()

Example 1: Vector input


AttributeError: 'TimeDependentRealNVP' object has no attribute 'eval'